In [1]:
!pip install -U bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 21.9 MB/s eta 0:00:00:00:0100:01


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments
import torch
from datasets import load_dataset


dataset = load_dataset("json", data_files="/kaggle/input/kjlkjkj/asd1.jsonl")

print(dataset)

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['question', 'answer'],
        num_rows: 1097
    })
})


In [3]:
import torch
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import (
    prepare_model_for_kbit_training,
    get_peft_model,
    LoraConfig,
    PeftModel
)
from datasets import load_dataset


model_name = "t-tech/T-lite-it-1.0"

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=True,
   bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map='auto',
    quantization_config=nf4_config,
    use_cache=False
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.33G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/4.90k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [4]:
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, peft_config)
tok = AutoTokenizer.from_pretrained(model_name)

In [5]:
def a(q):
    messages = [
    {"role": "user", "content": q}]
    text = tok.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    return text

In [6]:
def tokenize_function(examples):
    return tokenizer(
        [a(q) for q in examples["question"]],
        text_target=examples["answer"],
        padding="max_length",
        padding_side='left',
        truncation=True,
        max_length=64
    )


tokenized_dataset = dataset.map(tokenize_function, batched=True)

tokenized_dataset = tokenized_dataset.remove_columns(["question", "answer"])
print(tokenized_dataset)

Map:   0%|          | 0/1097 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1097
    })
})


In [7]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=10,
    save_strategy="steps",
    save_steps=500,
    report_to="none",
    fp16=True,
    gradient_checkpointing=True,
)

In [8]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
)

In [9]:
trainer.train()

Step,Training Loss
10,55.142100
20,44.821100
30,33.227600
40,29.486700
50,30.733600
60,28.233800
70,27.579200
80,26.163300
90,24.319200
100,23.197600


TrainOutput(global_step=204, training_loss=25.46945930929745, metrics={'train_runtime': 2010.9839, 'train_samples_per_second': 1.637, 'train_steps_per_second': 0.101, 'total_flos': 8888408909807616.0, 'train_loss': 25.46945930929745, 'epoch': 2.9890909090909092})

In [10]:
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json',
 './fine_tuned_model/tokenizer.json')

In [1]:
import torch
from peft import PeftModel    
from transformers import AutoModelForCausalLM, StoppingCriteria, AutoTokenizer, StoppingCriteriaList, TextIteratorStreamer

model_name = "t-tech/T-lite-it-1.0"
adapters_name = "/kaggle/input/dfdffd/asd1"

print(f"Starting to load the model {model_name} into memory")

m = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map={"": 0}
)
m = PeftModel.from_pretrained(m, adapters_name)
m = m.merge_and_unload()
tok = AutoTokenizer.from_pretrained(model_name)

print(f"Successfully loaded the model {model_name} into memory")

Starting to load the model t-tech/T-lite-it-1.0 into memory


config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.90k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

Successfully loaded the model t-tech/T-lite-it-1.0 into memory


In [39]:
prompt = "Как звали лучшего друга Пятачка?"
messages = [
    {"role": "user", "content": prompt}
]
text = tok.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tok([text], return_tensors="pt").to(m.device)

generated_ids = m.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tok.batch_decode(generated_ids, skip_special_tokens=True)[0]

print(response)

Лучшего друга Пятачка в мультфильме «Винни-Пух» зовут Кристофер Робин. Однако, если вы имеете в виду другого персонажа из мира Винни-Пуха, например, из книг А.А. Милна, то его другом является Тигруля (Тигра). Если речь идет о каком-то конкретном произведении или адаптации, уточните, пожалуйста.
